In [3]:
import re
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [2]:
class State:

    def __init__(self, t, u, d, b):
        self.t = t
        self.u = u
        self.d = d
        self.b = b
        
        assert len(u) == len(d[0]) == len(b)
        self.arm_rewards = np.dot(d,u)
    
    def __str__(self) -> str:
        return f"State(t={self.t}, u={self.u}, d={self.d}, b={self.b})"
    
    def __repr__(self) -> str:
        return f"State(t={self.t}, u={self.u}, d={self.d}, b={self.b})"
        
    def parse(s: str):
        t = int(re.search(r"\d+", s).group())
        u = np.array(eval(re.search(r"\[.*?\]", s).group()))
        d = np.array(eval(re.search(r"Array{Float64}(\[.*?\]\])", s).group(1)))
        b = np.array(eval(re.findall(r'\[.*?\]', s)[-1]))
        return State(t, u, d, b)
    
    def best_arm(self) -> str:
        return f'C{np.argmax(self.arm_rewards)+1}'

In [6]:
with open("../data/beliefs/test.json", "r") as f:
    data = json.load(f)

print(type(data))

<class 'dict'>


In [8]:
k1, k2 = data.keys()

In [15]:
exps = {}
exps["ai"] = ["active_infinite_23422_131424", "active_infinite_23419_153324", "active_infinite_23419_154231", "active_infinite_23419_155139", "active_infinite_23419_160549", "active_infinite_23419_161648", "active_infinite_23419_162359", "active_infinite_23419_163115", "active_infinite_23419_16401", "active_infinite_23419_164925", "active_infinite_23419_17049", "active_infinite_23419_171313", "active_infinite_23419_172230", "active_infinite_23419_172929", "active_infinite_23419_17394", "active_infinite_23419_174759", "active_infinite_23419_180013", "active_infinite_23419_181126", "active_infinite_23419_182029", "active_infinite_23419_183010"]
exps["af"] = ["active_finite_23419_214856", "active_finite_23419_22017", "active_finite_23419_22124", "active_finite_23419_222216", "active_finite_23419_223627", "active_finite_23419_22505", "active_finite_23419_230052", "active_finite_23419_230927", "active_finite_23419_232011", "active_finite_23419_233116", "active_finite_23419_23426", "active_finite_23419_235054", "active_finite_23420_000135", "active_finite_23420_001238", "active_finite_23420_002515", "active_finite_23420_004012", "active_finite_23420_00540", "active_finite_23420_010552", "active_finite_23420_011651", "active_finite_23420_012749"]
exps["pi"] = ["passive_infinite_23419_183940", "passive_infinite_23419_184948", "passive_infinite_23419_18573", "passive_infinite_23419_191144", "passive_infinite_23419_193217", "passive_infinite_23419_194023", "passive_infinite_23419_194838", "passive_infinite_23419_195741", "passive_infinite_23419_200444", "passive_infinite_23419_201343", "passive_infinite_23419_202155", "passive_infinite_23419_203025", "passive_infinite_23419_203934", "passive_infinite_23419_205227", "passive_infinite_23419_205917", "passive_infinite_23419_210814", "passive_infinite_23419_211748", "passive_infinite_23419_21258", "passive_infinite_23419_213346", "passive_infinite_23419_214029"]
exps["pf"] = ["passive_finite_23422_141255", "passive_finite_23422_142713", "passive_finite_23422_143917", "passive_finite_23422_145237", "passive_finite_23422_150344", "passive_finite_23422_151540", "passive_finite_23422_152710", "passive_finite_23422_153914", "passive_finite_23422_155019", "passive_finite_23422_16003", "passive_finite_23422_161248", "passive_finite_23422_162128", "passive_finite_23422_163156", "passive_finite_23422_164250", "passive_finite_23422_165540", "passive_finite_23422_17041", "passive_finite_23422_17148", "passive_finite_23422_172518", "passive_finite_23422_173548", "passive_finite_23422_174423"]



In [53]:
data_raw = {}
# for exp in exps.keys():
for exp in ["ai"]:
    data_raw[exp] = {}
    for id in exps[exp][:2]:
        with open("../data/beliefs/"+id+".json", "r") as f:
            # list{list{dict{string:float}}}
            data_raw[exp][id] = json.load(f)

# dict{string:dict{string:list{list{dict{string:float}}}}}
# dimensions: exp(4):(id(20):(runs(25),steps(1000)(state:probability))
print(type(data_raw))

<class 'dict'>


In [61]:
w = list(data_raw["ai"]["active_infinite_23422_131424"][0][-1].keys())[0]
e = data_raw["ai"]["active_infinite_23422_131424"][0][-1]

f = parse_states_in_dict(e)
f

{State(t=1, u=[8. 3. 3.], d=[[0.5 0.  0.5]
  [0.  0.5 0.5]
  [0.  0.5 0.5]], b=[0.e+00 1.e-02 5.e+01]): 0.5969999999999506,
 State(t=1, u=[ 3. -2. -2.], d=[[0.5 0.  0.5]
  [0.  0.5 0.5]
  [0.  0.5 0.5]], b=[0.e+00 1.e-02 5.e+01]): 0.26499999999998713,
 State(t=1, u=[ 8. -2. -2.], d=[[0.5 0.  0.5]
  [0.  0.5 0.5]
  [0.  0.5 0.5]], b=[0.e+00 1.e-02 5.e+01]): 0.13800000000000112}

In [73]:
def parse_states_in_dict(d: dict):
    d_new = {}
    for s in d.keys():
        d_new[State.parse(s)] = d[s]
    return d_new

def parse_states_in_dict_from_list(d: list[dict]):
    return [parse_states_in_dict(x) for x in d]

data = {}
for exp in data_raw.keys():
    data[exp] = {}
    for id in data_raw[exp].keys():
        # should be 2d array of dicts
        dicts_raw = data_raw[exp][id]
        dicts = map(parse_states_in_dict_from_list, dicts_raw)
        data[exp][id] = dicts

In [74]:
list(data["ai"]["active_infinite_23422_131424"])

[[{State(t=1000, u=[-2.  8. -2.], d=[[0.5 0.  0.5]
    [0.5 0.5 0. ]
    [1.  0.  0. ]], b=[0.e+00 1.e-02 5.e+01]): 0.0001,
   State(t=1000, u=[-2.  3.  8.], d=[[1.  0.  0. ]
    [0.5 0.5 0. ]
    [0.  0.5 0.5]], b=[0.e+00 1.e-02 5.e+01]): 0.0004,
   State(t=1000, u=[ 8. -2. -2.], d=[[0.5 0.  0.5]
    [0.  0.5 0.5]
    [0.5 0.5 0. ]], b=[0.e+00 1.e-02 5.e+01]): 0.0002,
   State(t=1000, u=[-2.  3. -2.], d=[[1.  0.  0. ]
    [0.  1.  0. ]
    [0.5 0.  0.5]], b=[0.e+00 1.e-02 5.e+01]): 0.0001,
   State(t=1000, u=[3. 8. 8.], d=[[1. 0. 0.]
    [0. 0. 1.]
    [0. 1. 0.]], b=[0.e+00 1.e-02 5.e+01]): 0.0001,
   State(t=1000, u=[ 8. -2. -2.], d=[[0.5 0.  0.5]
    [0.  0.  1. ]
    [0.  1.  0. ]], b=[0.e+00 1.e-02 5.e+01]): 0.0002,
   State(t=1000, u=[3. 8. 3.], d=[[0.5 0.  0.5]
    [0.5 0.  0.5]
    [1.  0.  0. ]], b=[0.e+00 1.e-02 5.e+01]): 0.0001,
   State(t=1000, u=[ 8. -2. -2.], d=[[0.5 0.  0.5]
    [0.  0.5 0.5]
    [0.5 0.  0.5]], b=[0.e+00 1.e-02 5.e+01]): 0.0001,
   State(t=1000, u=[-2.

In [ ]:
# TODO - calc "converged" state for each run

In [ ]:
# TODO - collect "converged" state probs as np array (ids, exps, steps)